In [2]:
import pytest
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('Downloads/creditcard_2023.csv')

@pytest.fixture
def sample_data():
    data = {
        'Feature1': np.random.rand(100),
        'Feature2': np.random.rand(100),
        'Class': np.random.randint(0, 2, 100)
    }
    df = pd.DataFrame(data)
    return df

# Test full pipeline integration
def test_pipeline_integration(sample_data):
    # Preprocessing
    X = sample_data.drop(columns='Class', axis=1)
    Y = sample_data['Class']

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

    # Model Training
    sgd_model = SGDClassifier(loss='log_loss', max_iter=10000, random_state=42)
    adaboost_model = AdaBoostClassifier(n_estimators=50, random_state=42)

    sgd_model.fit(X_train, Y_train)
    adaboost_model.fit(X_train, Y_train)

    # Predictions
    sgd_preds = sgd_model.predict_proba(X_test)[:, 1]
    adaboost_preds = adaboost_model.predict_proba(X_test)[:, 1]

    hybrid_preds = (0.5 * sgd_preds) + (0.5 * adaboost_preds)
    hybrid_preds_class = (hybrid_preds >= 0.5).astype(int)

    # Accuracy Calculation
    test_accuracy = accuracy_score(Y_test, hybrid_preds_class)

    assert test_accuracy > 0  # Ensure model is making valid predictions
